<a href="https://colab.research.google.com/github/s0ye0nyang/News-Recomendation-System/blob/main/Preprocessing_and_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **데이터 전처리**

In [2]:
#Konlpy 사용 위해
!apt-get update 
!apt-get install g++ openjdk-8-jdk python-dev python3-dev 
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk-13.0.2"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [223 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-securit

In [3]:
from tqdm import tqdm_notebook
from konlpy.tag import Komoran
import numpy as np
import string
import re
import warnings
import pickle
from gensim import corpora
from gensim import models
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [33]:
# 저장했던 기사 읽기
def read_documents(input_file_name):
  corpus = []

  with open(input_file_name, 'rb') as f:
    temp_corpus = pickl.load(f)

  for page in temp_corpus:
    corpus+=page

  return corpus
    

# 내용 중 한국어를 제외한 글자들을 제거
def text_cleaning(docs):
  for doc in docs:
    doc=re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "", doc)

  return docs


# 불용어 데이터를 읽어와 불용어들을 제거
# string.puctuation을 불용어에 추가하여 특수문자도 제거!
def define_stopwords(path):
  SW = set()

  for i in string.punctuation:
    SW.add(i)

  with open(path, encoding='utf-8') as f:
    for word in f:
      SW.add(word)

  return SW


# 명사 추출과 형태소 분석 
def text_tokenizing(corpus, tokenizer):
  komoran = Komoran()
  token_corpus = []

  if tokenizer=="noun": # 명사
    for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
      token_text = komoran.nouns(corpus[n])
      token_text = [word for word in token_text if word not in SW and len(word)>1]
      token_corpus.append(token_text)
  
  elif tokenizer=="morph": # 형태소
    for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
      token_text = komoran.morphs(corpus[n])
      token_text = [word for word in token_text if word not in SW and len(word)>1]
      token_corpus.append(token_text)

  elif tokenizer=="word": # 단어
    for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
      token_text = corpus[n].split()
      token_text = [word for word in token_text if word not in SW and len(word)>1]
      token_corpus.append(token_text)

  return token_corpus


In [52]:
#input_file_name = "input.txt"
#documents = read_documents(input_file_name)
data = open('input.txt', 'r', encoding='utf-8')
documents = data.readlines()
SW = define_stopwords("stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun" or "word")#"noun" or "word"

In [65]:
print(tokenized_text[0])
# tokenized_text[1]은 공백
print(tokenized_text[2])

['문재인', '대통령', '국회', '본회의', '야당', '거부권', '무력', '고위', '공직자', '범죄', '수사', '공수', '개정안', '통과', '관련', '공수', '처장', '후보', '추천', '임명', '청문회', '나머지', '절차', '신속', '차질', '진행', '새해', '벽두', '공수', '처가', '정식', '출범', '기대', '대통령', '공수', '개정안', '통과', '직후', '기약', '공수', '출범', '법안', '개정', '신속', '출범', '다행', '입장', '강민석', '청와대', '대변인', '개정', '완료', '만큼', '공수', '처장', '인선', '절차', '완료', '내년', '공수', '처가', '정식', '출범', '사실', '가이드라인', '풀이', '윤석열', '검찰', '총장', '징계', '논의', '법무부', '검사', '징계', '위원회', '소집', '대통령', '검찰', '개혁', '제도', '완수', '정치', '의미', '부여', '대통령', '청와대', '수석', '보좌관', '회의', '공수', '처리', '한국', '민주주의', '장이', '역사', '시간', '평가', '대통령', '공수', '설치', '대통령', '권력', '비리', '성역', '수사', '사정', '권력', '기관', '사이', '견제', '균형', '부패', '사회', '숙원', '국민', '약속', '평가', '공수', '처가', '정권', '보호', '우려', '권력', '비리', '수사', '기관', '강조', '공수', '설치', '기능', '생각', '원래', '야당', '적극', '여당', '소극', '논의', '고도', '7월', '공수', '시행', '이후', '공수', '처장', '후보', '추천', '위원회', '구성', '후보', '추천', '협조', '공수', '개정', '반대', '필리버스터', '의사', '진행', '방해', '진행', '야당', '직접', '비

In [63]:
# 토픽 모델링
dictionary = corpora.Dictionary(tokenized_text)
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

print(dictionary)

Dictionary(256 unique tokens: ['7월', '가동', '가이드라인', '각인', '감사']...)


In [69]:
# LDA 모델
model = models.ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary)

model.show_topic(1,30)

[('공수', 0.039703447),
 ('대통령', 0.023589494),
 ('신혼부부', 0.01493309),
 ('권력', 0.011740992),
 ('처가', 0.010103763),
 ('부부', 0.009961555),
 ('출범', 0.009799224),
 ('설치', 0.00961475),
 ('대표', 0.009064027),
 ('수사', 0.008816581),
 ('야당', 0.008811099),
 ('통과', 0.0077159675),
 ('숙원', 0.007687227),
 ('진행', 0.00766414),
 ('당원', 0.0076606325),
 ('개정안', 0.007458982),
 ('개혁', 0.0070253965),
 ('추천', 0.0068664136),
 ('기관', 0.006854698),
 ('개정', 0.006624617),
 ('처장', 0.006602897),
 ('후보', 0.006440442),
 ('만원', 0.006401433),
 ('미만', 0.006063924),
 ('방해', 0.0059653753),
 ('처리', 0.005949876),
 ('지난해', 0.005828304),
 ('여당', 0.0057756463),
 ('대출', 0.0056646094),
 ('징계', 0.0056295698)]